In [1]:
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter


# [START trace_setting]
from azure.core.settings import settings

settings.tracing_implementation = "opentelemetry"
# [END trace_setting]

# [START instrument_inferencing]
from azure.ai.inference.tracing import AIInferenceInstrumentor

# Instrument AI Inference API
AIInferenceInstrumentor().instrument()
# [END instrument_inferencing]

# Select Model

In [2]:
models = {
    "gpt-4o": "gpt-4o",
    "DeepSeek-R1": "DeepSeek-R1",
    "Phi-4-mini-instruct": "Phi-4-mini-instruct",
    "Phi-4-multimodal-instruct (version:1)": "Phi-4-multimodal-instruct",

    "o3-mini": "o3-mini",
}

# Display available models
print("Available models:")
for i, model_name in enumerate(models.keys()):
    print(f"{i+1}. {model_name}")

# Prompt user for selection
selection = input("Select a model (enter number): ")
try:
    selection_idx = int(selection) - 1
    model_keys = list(models.keys())
    if 0 <= selection_idx < len(model_keys):
        selected_model = models[model_keys[selection_idx]]
        print(f"Selected model: {selected_model}")
    else:
        print("Invalid selection. Using default model 'gpt-4o'")
        selected_model = "gpt-4o"
except ValueError:
    print("Invalid input. Using default model 'gpt-4o'")
    selected_model = "gpt-4o"

Available models:
1. gpt-4o
2. DeepSeek-R1
3. Phi-4-mini-instruct
4. Phi-4-multimodal-instruct (version:1)
5. o3-mini
Selected model: gpt-4o


# Loading Libraries

In [3]:
import os
from geopy.geocoders import Nominatim
import requests
from openai import OpenAI


In [4]:
from azure.ai.inference import ChatCompletionsClient
from azure.core.credentials import AzureKeyCredential

from azure.identity import DefaultAzureCredential, get_bearer_token_provider, AzureCliCredential



In [5]:
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    ToolMessage,
    AssistantMessage
)

# Initilise a Chat Completion Client

In [6]:
# endpoint = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT")
# inferencekey = os.getenv("AZURE_INFERENCE_SDK_ENDPOINT_KEY")


endpoint = os.getenv("GITHUB_INFERENCE_ENDPOINT")
inferencekey = os.getenv("GITHUB_TOKEN")

In [7]:
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(inferencekey),
    credential_scopes=["https://cognitiveservices.azure.com/.default"],

    )

## Run a basic Chat Completion test

In [8]:
messages = [
    UserMessage(content="1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?"),
]

In [9]:
print(messages[0].content)
# gpt-4o
response = client.complete(
    messages = messages,
    model = selected_model
)
print(response.choices[0].message.content)

1 shirt needs 1 hour to dry, how long does it take to dry 3 shirts?
It would still take **1 hour** to dry 3 shirts, assuming they are being dried under the same conditions (e.g., they are dried in parallel, not one after the other). Drying multiple shirts simultaneously doesn't increase the drying time if all are exposed to the same drying conditions at the same time.


# Tool Calling

![Tool calling flow](https://techcommunity.microsoft.com/t5/s/gxcuf89792/images/bS00MzkxMDI5LUM2YTczNA?revision=7&image-dimensions=2000x2000&constrain-image=true "Tool calling flow")


Using tools requires the following parts 
- Tool definition using `ChatCompletionsToolDefinition` and `FunctionDefinition`
- Tool function, e.g. `get_weather`
- Tool message holding the result of a tool call, using `ToolMessage`

The process to perform a tool call: 
1. Define the tool using `ChatCompletionsToolDefinition` and create a function 

2. Create a chat completion request and pass an array of defined tools 
   1. append response message to chat history (`messages`)

3. Inspect the response from the chat completion for existence of tool calls, `response.choices[0].message.tool_calls`
   1. In case a tool call existes in response, extract the tool call details (function name, arguments, tool call id) and call the function
   2. Create a `ToolMessage` object with tool call id and function result as content
   3. Append the `ToolMessage` to the chat history (`messages`)

4. Send the updated chat history to the chat completion API again

references:
- https://learn.microsoft.com/en-us/azure/ai-foundry/model-inference/how-to/use-chat-completions?pivots=programming-language-python#use-tools

## Essential lirbaries for Tool Calling

In [10]:
import json
import logging

from azure.ai.inference.models import (
    FunctionDefinition,
    ChatCompletionsToolDefinition,

    SystemMessage,
    UserMessage,
    ToolMessage,

    CompletionsFinishReason,

)

from azure.ai.inference import (
    ChatCompletionsClient
)

## Tool Functions

This section shows the function definition and the function decleration itself.


This schema represents the function signature and the parameters of the function. The function itself is defined in the next section.

```python
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city name, e.g. San Francisco",
                    },
                },
                "required": ["location"],
            },
        }
    }
]
```

Tools will be defined using SDK classes

In [11]:
func_def_get_weather = FunctionDefinition(
    name="get_weather",
    description="Get the weather for a given city",
    parameters={
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The city to get the weather for"
            },
            "unit": {
                "type": "string",
                "description": "The unit to use for the temperature (C or F)"
            }
        },
        "required": ["city"]
    }
)

func_def_get_current_time = FunctionDefinition(
    name="get_current_time",
    description="Get the current time in a given location",
    parameters={
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city name, e.g. San Francisco"
            }
        },
        "required": ["location"]
    }
)


tools = [
    ChatCompletionsToolDefinition(function=func_def_get_weather),
    ChatCompletionsToolDefinition(function=func_def_get_current_time)
]

In [12]:

def get_weather(city, unit="celsius"):


    """Get the coordinates of a location using Nominatim."""
    geolocator = Nominatim(user_agent="my_user_agent")
    location = geolocator.geocode(city)

    """Get the weather for a given city"""
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={location.latitude}&longitude={location.longitude}&temperature_unit{unit}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()


    print(f"get_weather called with city: {city}, unit: {unit}")

    # Simulate getting weather data
    weather_data = {
        "city": city,
        "temperature": data['current']['temperature_2m'],
        "unit": unit,
        "description": "no description"
    }
    return json.dumps(weather_data)

def get_current_time(location):
    from datetime import datetime
    """Get the current time for a given location"""
    print(f"get_current_time called with location: {location}")
    location_lower = location.lower()


    current_time = datetime.now().strftime("%I:%M %p")
    return json.dumps({
        "location": location,
        "current_time": current_time
    })

    print(f"No timezone data found for {location_lower}")
    return json.dumps({"location": location, "current_time": "unknown"})

# print(get_weather("Sydney", "C"))
# print(get_current_time("Sydney"))

## The first call 

In the first call we pass th user message in the message history, and the tool definition in the tools parameter.

Must inspect the finish reason in the response. If the finish reason is `TOOL_CALLS` then we need to call the function and pass the result back to the chat completion API.

In [13]:
message_history = [
    SystemMessage(content="You are a helpful assistant."),
    UserMessage(content="What is the time and temperature now in Melbourne")
]


first_response = client.complete(
    messages=message_history,
    model = selected_model,
    tools=tools,
    tool_choice="auto"
)

#inspect first_response
print("finish reason: ", first_response.choices[0].finish_reason)
first_response.choices[0].message.as_dict()

finish reason:  CompletionsFinishReason.TOOL_CALLS


{'content': None,
 'refusal': None,
 'role': 'assistant',
 'tool_calls': [{'function': {'arguments': '{"location": "Melbourne"}',
    'name': 'get_current_time'},
   'id': 'call_gBHqGnNGvDTSHA1GnbrWbFfq',
   'type': 'function'},
  {'function': {'arguments': '{"city": "Melbourne", "unit": "C"}',
    'name': 'get_weather'},
   'id': 'call_afakH4HwGmPbbYckGqRlnpbV',
   'type': 'function'}]}

### Finish reasons

`CompletionsFinishReason` is an enum that defines the possible reasons for a completion to finish.


The enum values are:
```python
CompletionsFinishReason.CONTENT_FILTERED
CompletionsFinishReason.STOPPED
CompletionsFinishReason.TOOL_CALLS
CompletionsFinishReason.TOKEN_LIMIT_REACHED
```

In [14]:
finish_reason = first_response.choices[0].finish_reason

response_message = first_response.choices[0].message
tool_calls = response_message.tool_calls if finish_reason == CompletionsFinishReason.TOOL_CALLS else []

## Perform Tool Calls

In [15]:
# append response_message to chat history and perform tool calls
message_history.append(response_message)

In [16]:
message_history

[{'role': 'system', 'content': 'You are a helpful assistant.'},
 {'role': 'user', 'content': 'What is the time and temperature now in Melbourne'},
 {'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"location": "Melbourne"}', 'name': 'get_current_time'}, 'id': 'call_gBHqGnNGvDTSHA1GnbrWbFfq', 'type': 'function'}, {'function': {'arguments': '{"city": "Melbourne", "unit": "C"}', 'name': 'get_weather'}, 'id': 'call_afakH4HwGmPbbYckGqRlnpbV', 'type': 'function'}]}]

### Executing function calls using locals() and dictionary unpacking with ** 


All the function calls are equivalent to each other. 


```python
function_name= "get_weather"
function_args = {
    "city": "Sydney",
    "unit": "C"
}

locals()[function_name](**function_args)
# equivalant to
locals()['get_weather'](**{'city': 'Sydney', 'unit': 'C'})\
# and
get_weather(city="Sydney", unit="C")
# and
get_weather(**{"city": "Sydney", "unit": "C"})
```

and will produce the same output

In [17]:
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    tool_call_id = tool_call.id



    # execute the function call and append the results as a ToolMessage in the message history
    function_call_results = locals()[function_name](**function_args)

    message_history.append(
        ToolMessage(
            tool_call_id=tool_call_id,
            content=function_call_results
        )
    )


    print(f"Tool Call ID: {tool_call_id}")
    print(f"Function: {function_name}")
    print(f"Arguments: {json.dumps(function_args, indent=2)}")
    print(f"Results: {function_call_results}")
    print("---")



get_current_time called with location: Melbourne
Tool Call ID: call_gBHqGnNGvDTSHA1GnbrWbFfq
Function: get_current_time
Arguments: {
  "location": "Melbourne"
}
Results: {"location": "Melbourne", "current_time": "06:31 PM"}
---
get_weather called with city: Melbourne, unit: C
Tool Call ID: call_afakH4HwGmPbbYckGqRlnpbV
Function: get_weather
Arguments: {
  "city": "Melbourne",
  "unit": "C"
}
Results: {"city": "Melbourne", "temperature": 17.2, "unit": "C", "description": "no description"}
---


In [18]:
## inspect message history after adding tool call results
for message in message_history:
    print(message.as_dict())
    print("---")

{'role': 'system', 'content': 'You are a helpful assistant.'}
---
{'role': 'user', 'content': 'What is the time and temperature now in Melbourne'}
---
{'content': None, 'refusal': None, 'role': 'assistant', 'tool_calls': [{'function': {'arguments': '{"location": "Melbourne"}', 'name': 'get_current_time'}, 'id': 'call_gBHqGnNGvDTSHA1GnbrWbFfq', 'type': 'function'}, {'function': {'arguments': '{"city": "Melbourne", "unit": "C"}', 'name': 'get_weather'}, 'id': 'call_afakH4HwGmPbbYckGqRlnpbV', 'type': 'function'}]}
---
{'role': 'tool', 'tool_call_id': 'call_gBHqGnNGvDTSHA1GnbrWbFfq', 'content': '{"location": "Melbourne", "current_time": "06:31 PM"}'}
---
{'role': 'tool', 'tool_call_id': 'call_afakH4HwGmPbbYckGqRlnpbV', 'content': '{"city": "Melbourne", "temperature": 17.2, "unit": "C", "description": "no description"}'}
---


## Send results of Tool Calls

after the function call is executed, we need to send the result back to the chat completion API.

In [19]:
if finish_reason == CompletionsFinishReason.TOOL_CALLS:
    final_response = client.complete(
        messages=message_history,
        model = selected_model,
        tools=tools,
        tool_choice="auto"
    )

    print("Final Response: ", final_response.choices[0].message.content)


Final Response:  Currently, in Melbourne, the time is 6:31 PM, and the temperature is 17.2°C.


In [20]:
# [START uninstrument_inferencing]
AIInferenceInstrumentor().uninstrument()
# [END uninstrument_inferencing]

In [21]:
os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED'] = 'true'


In [22]:
os.environ['AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED']

'true'